In [ ]:
# =========================================
# 1. Librerías
# =========================================
import paho.mqtt.client as mqtt
import torch
import torch.nn as nn
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =========================================
# 2. Definir modelo (debe coincidir con el entrenado)
# =========================================
class CNNLSTM(nn.Module):
    def __init__(self, input_dim=6, cnn_channels=64, lstm_hidden=256, lstm_layers=2, dropout=0.7):
        super().__init__()
        self.conv1 = nn.Conv1d(input_dim, cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(cnn_channels, lstm_hidden, lstm_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(lstm_hidden*2, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.transpose(1, 2)       # (batch, features, seq)
        x = self.relu(self.conv1(x))
        x = x.transpose(1, 2)       # (batch, seq, features)
        out, _ = self.lstm(x)
        out = out[:, -1, :]         # último estado
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out).squeeze(-1)

# =========================================
# 3. Cargar modelo entrenado
# =========================================
model = CNNLSTM(input_dim=6).to(device)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()
print("✅ Modelo cargado correctamente")

# =========================================
# 4. Callback para procesar mensajes MQTT
# =========================================
import json
import torch

THRESHOLD = 0.5  # ajusta según tu entrenamiento

def on_message(client, userdata, msg):
    try:

        # Decodificar mensaje JSON del ESP8266
        data = json.loads(msg.payload.decode())
        print(f"📩 Datos recibidos: {data}")

        # Extraer features (6 valores usados en el modelo)
        features = [data["ax"], data["ay"], data["az"],
                    data["gx"], data["gy"], data["gz"]]

        # Crear tensor (batch=1, seq_len=1, features=6)
        x = torch.tensor(features, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

        # Predicción con el modelo
        with torch.no_grad():
            output = model(x)
            prob = torch.sigmoid(output).item()
            pred = 1 if prob > THRESHOLD else 0

        print(f"🤖 Prob: {prob:.4f} → {'⚠️ Caída detectada' if pred==1 else '✅ Sin caída'}")

    except Exception as e:
        print(f"❌ Error procesando mensaje: {e}")


✅ Modelo cargado correctamente


In [ ]:
# =========================================
# 5. Conectar al broker HiveMQ Cloud
# =========================================
broker = "45abc320943e4d708930117ef02e02c5.s1.eu.hivemq.cloud"
port = 8883
username = "esp8266"
password = "Giorpa469"
topic = "esp8266/mpu6050"   # <-- asegúrate de que sea el mismo que usas en tu ESP8266

client = mqtt.Client()
client.username_pw_set(username, password)
client.tls_set()   # HiveMQ Cloud exige SSL

client.on_message = on_message

client.connect(broker, port, 60)
client.subscribe(topic)

print(f"📡 Suscrito a {topic} en {broker}:{port}")
client.loop_forever()


NameError: name 'mqtt' is not defined

In [ ]:
# =========================================
# 1. Librerías
# =========================================
import paho.mqtt.client as mqtt
import torch
import torch.nn as nn
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =========================================
# 2. Definir modelo (debe coincidir con el entrenado)
# =========================================
class CNNLSTM(nn.Module):
    def __init__(self, input_dim=6, cnn_channels=64, lstm_hidden=256, lstm_layers=2, dropout=0.7):
        super().__init__()
        self.conv1 = nn.Conv1d(input_dim, cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(cnn_channels, lstm_hidden, lstm_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(lstm_hidden*2, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.transpose(1, 2)       # (batch, features, seq)
        x = self.relu(self.conv1(x))
        x = x.transpose(1, 2)       # (batch, seq, features)
        out, _ = self.lstm(x)
        out = out[:, -1, :]         # último estado
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out).squeeze(-1)

# =========================================
# 3. Cargar modelo entrenado
# =========================================
model = CNNLSTM(input_dim=6).to(device)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()
print("✅ Modelo cargado correctamente")

# =========================================
# 4. Callback para procesar mensajes MQTT
# =========================================
import json
import torch

THRESHOLD = 0.5  # ajusta según tu entrenamiento

def on_message(client, userdata, msg):
    try:
        # Decodificar mensaje JSON del ESP8266
        data = json.loads(msg.payload.decode())
        print(f"📩 Datos recibidos: {data}")

        # Extraer features (6 valores usados en el modelo)
        features = [data["ax"], data["ay"], data["az"],
                    data["gx"], data["gy"], data["gz"]]

        # Crear tensor (batch=1, seq_len=1, features=6)
        x = torch.tensor(features, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

        # Predicción con el modelo
        with torch.no_grad():
            output = model(x)
            prob = torch.sigmoid(output).item()
            pred = 1 if prob > THRESHOLD else 0

        print(f"🤖 Prob: {prob:.4f} → {'⚠️ Caída detectada' if pred==1 else '✅ Sin caída'}")

    except Exception as e:
        print(f"❌ Error procesando mensaje: {e}")

# =========================================
# 5. Conectar al broker HiveMQ Cloud
# =========================================
broker = "45abc320943e4d708930117ef02e02c5.s1.eu.hivemq.cloud"
port = 8883
username = "esp8266"
password = "Giorpa469"
topic = "esp8266/mpu6050"   # <-- asegúrate de que sea el mismo que usas en tu ESP8266

client = mqtt.Client()
client.username_pw_set(username, password)
client.tls_set()   # HiveMQ Cloud exige SSL

client.on_message = on_message

client.connect(broker, port, 60)
client.subscribe(topic)

print(f"📡 Suscrito a {topic} en {broker}:{port}")
client.loop_forever()


✅ Modelo cargado correctamente


/tmp/ipython-input-311304524.py:83: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


📡 Suscrito a esp8266/mpu6050 en 45abc320943e4d708930117ef02e02c5.s1.eu.hivemq.cloud:8883
📩 Datos recibidos: {'ax': 0.5722141862, 'ay': 0.0023942017, 'az': 9.7324295044, 'gx': -0.0381041355, 'gy': -0.0087932618, 'gz': -0.0127901994, 'temp': 24.6652946472, 'fecha': '2025-09-16 16:59:27'}
🤖 Prob: 0.6148 → ⚠️ Caída detectada
📩 Datos recibidos: {'ax': 0.5674257874, 'ay': -0.0143652102, 'az': 9.7348241806, 'gx': -0.0383705944, 'gy': -0.0085267993, 'gz': -0.0127901994, 'temp': 24.6476478577, 'fecha': '2025-09-16 16:59:28'}
🤖 Prob: 0.6151 → ⚠️ Caída detectada
📩 Datos recibidos: {'ax': 0.5650315881, 'ay': -0.0095768068, 'az': 9.6965169907, 'gx': -0.0375712104, 'gy': -0.0098591121, 'gz': -0.0125237368, 'temp': 24.6623535156, 'fecha': '2025-09-16 16:59:28'}
🤖 Prob: 0.6143 → ⚠️ Caída detectada
📩 Datos recibidos: {'ax': 0.5650315881, 'ay': -0.0095768068, 'az': 9.6965169907, 'gx': -0.0375712104, 'gy': -0.009326187, 'gz': -0.0119908117, 'temp': 24.6711769104, 'fecha': '2025-09-16 16:59:29'}
🤖 Prob: 0

KeyboardInterrupt: 

In [ ]:
# =========================================
# 5. Conectar al broker HiveMQ Cloud
# =========================================
broker = "45abc320943e4d708930117ef02e02c5.s1.eu.hivemq.cloud"
port = 8883
username = "esp8266"
password = "Giorpa469"
topic = "esp8266/mpu6050"   # <-- asegúrate de que sea el mismo que usas en tu ESP8266

client = mqtt.Client()
client.username_pw_set(username, password)
client.tls_set()   # HiveMQ Cloud exige SSL

client.on_message = on_message

client.connect(broker, port, 60)
client.subscribe(topic)

print(f"📡 Suscrito a {topic} en {broker}:{port}")
client.loop_forever()


/tmp/ipython-input-2144895886.py:10: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Se truncaron las últimas líneas 5000 del resultado de transmisión.
📩 Datos recibidos: {'ax': 0.0, 'ay': 0.0, 'az': 122.5831298828, 'gx': -6.8759584427, 'gy': 1.9070914984, 'gz': 19.8871765137, 'temp': 36.5299987793, 'fecha': '2025-09-16 16:35:46'}
🤖 Prob: 0.9156 → ⚠️ Caída detectada
📩 Datos recibidos: {'ax': 0.0, 'ay': 0.0, 'az': 122.5831298828, 'gx': 15.738825798, 'gy': 1.9070914984, 'gz': 19.8871765137, 'temp': 36.5299987793, 'fecha': '2025-09-16 16:35:47'}
🤖 Prob: 0.8840 → ⚠️ Caída detectada
📩 Datos recibidos: {'ax': 0.0, 'ay': 0.0, 'az': 122.5831298828, 'gx': -28.3956565857, 'gy': 1.9070914984, 'gz': 19.8871765137, 'temp': 36.5299987793, 'fecha': '2025-09-16 16:35:47'}
🤖 Prob: 0.9384 → ⚠️ Caída detectada
📩 Datos recibidos: {'ax': 0.0, 'ay': 0.0, 'az': 122.5831298828, 'gx': -7.4165849686, 'gy': 1.9070914984, 'gz': 19.8871765137, 'temp': 36.5299987793, 'fecha': '2025-09-16 16:35:47'}
🤖 Prob: 0.9164 → ⚠️ Caída detectada
📩 Datos recibidos: {'ax': 0.0, 'ay': 0.0, 'az': 122.5831298828, '

KeyboardInterrupt: 

In [ ]:
import json
import paho.mqtt.client as mqtt

# ==========================
# CONFIGURACIÓN
# ==========================
BROKER = "45abc320943e4d708930117ef02e02c5.s1.eu.hivemq.cloud"     # Broker público, puedes usar tu propio broker
PUERTO = 8883
TOPIC = "esp8266/mpu6050"    # Topic donde el ESP8266 publica los datos

UMBRAL_CAIDA = 0.7               # Probabilidad mínima para considerar caída
VENTANA = 3                      # Número de lecturas consecutivas para confirmar caída

# ==========================
# VARIABLES GLOBALES
# ==========================
contador_caida = 0

# ==========================
# CALLBACKS MQTT
# ==========================
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("✅ Conectado al broker MQTT")
        client.subscribe(TOPIC)
        print(f"📡 Suscrito al topic: {TOPIC}")
    else:
        print(f"❌ Error al conectar, código: {rc}")

def on_message(client, userdata, msg):
    global contador_caida
    try:
        # Convertir el mensaje de JSON
        dato = json.loads(msg.payload.decode())
        prob = float(dato.get('probabilidad', 0))

        # Filtrado por ventana
        if prob >= UMBRAL_CAIDA:
            contador_caida += 1
            if contador_caida >= VENTANA:
                print(f"⚠️ CAÍDA CONFIRMADA! Fecha: {dato.get('fecha','N/A')} Prob: {prob}")
                contador_caida = 0  # Reiniciar contador después de confirmar
        else:
            contador_caida = 0  # Reiniciar si la probabilidad baja

    except Exception as e:
        print(f"❌ Error procesando el mensaje: {e}")

# ==========================
# CONEXIÓN MQTT
# ==========================
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(BROKER, PUERTO, 60)

# ==========================
# BUCLE PRINCIPAL
# ==========================
try:
    print("🔄 Esperando datos del ESP8266...")
    client.loop_forever()
except KeyboardInterrupt:
    print("🛑 Conexión finalizada por el usuario")
    client.disconnect()


ModuleNotFoundError: No module named 'paho'

In [ ]:
!pip install paho-mqtt torch
from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 1.3 MB/s eta 0:00:00


In [ ]:
  # =========================================
# 1. Librerías
# =========================================
import paho.mqtt.client as mqtt
import torch
import torch.nn as nn
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# =========================================
# 2. Definir modelo (debe coincidir con el entrenado)
# =========================================
class CNNLSTM(nn.Module):
    def __init__(self, input_dim=6, cnn_channels=64, lstm_hidden=256, lstm_layers=2, dropout=0.7):
        super().__init__()
        self.conv1 = nn.Conv1d(input_dim, cnn_channels, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.lstm = nn.LSTM(cnn_channels, lstm_hidden, lstm_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(lstm_hidden*2, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = x.transpose(1, 2)       # (batch, features, seq)
        x = self.relu(self.conv1(x))
        x = x.transpose(1, 2)       # (batch, seq, features)
        out, _ = self.lstm(x)
        out = out[:, -1, :]         # último estado
        out = self.relu(self.fc1(out))
        out = self.dropout(out)
        return self.fc2(out).squeeze(-1)

# =========================================
# 3. Cargar modelo entrenado
# =========================================
model = CNNLSTM(input_dim=6).to(device)
model.load_state_dict(torch.load("best_model.pth", map_location=device))
model.eval()
print("✅ Modelo cargado correctamente")

# =========================================
# 4. Configuración MQTT y variables
# =========================================
BROKER = "12f114da5d6a4b0d93d624aa7d4982ee.s1.eu.hivemq.cloud"
PORT = 8883
USERNAME = "esp32"
PASSWORD = "Giorpa469"
TOPIC = "esp32/mpu02"

THRESHOLD = 0.5    # Probabilidad mínima para considerar caída
UMBRAL_CAIDA = 0.7 # Para ventana de filtrado
VENTANA = 3        # Lecturas consecutivas para confirmar caída

contador_caida = 0

# =========================================
# 5. Callback MQTT
# =========================================
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("✅ Conectado al broker MQTT")
        client.subscribe(TOPIC)
        print(f"📡 Suscrito al topic: {TOPIC}")
    else:
        print(f"❌ Error al conectar, código: {rc}")

def on_message(client, userdata, msg):
    global contador_caida
    try:
        # Decodificar mensaje JSON del ESP8266
        data = json.loads(msg.payload.decode())
        features = [data["ax"], data["ay"], data["az"], data["gx"], data["gy"], data["gz"]]
        x = torch.tensor(features, dtype=torch.float32).unsqueeze(0).unsqueeze(0).to(device)

        # Predicción con el modelo
        with torch.no_grad():
            output = model(x)
            prob = torch.sigmoid(output).item()
            pred = 1 if prob > THRESHOLD else 0

        # Filtrado por ventana para reducir falsos positivos
        if prob >= UMBRAL_CAIDA:
            contador_caida += 1
            if contador_caida >= VENTANA:
                estado = "⚠️ CAÍDA CONFIRMADA"
                contador_caida = 0
            else:
                estado = "⚠️ Posible caída"
        else:
            contador_caida = 0
            estado = "✅ Sin caída"

        # Mostrar resultado constante
        print(f"{data.get('fecha','N/A')} | Prob: {prob:.4f} | Estado: {estado}")

    except Exception as e:
        print(f"❌ Error procesando mensaje: {e}")

# =========================================
# 6. Conexión MQTT
# =========================================
client = mqtt.Client()
client.username_pw_set(USERNAME, PASSWORD)
client.tls_set()  # SSL requerido por HiveMQ Cloud
client.on_connect = on_connect
client.on_message = on_message

client.connect(BROKER, PORT, 60)

# =========================================
# 7. Bucle principal
# =========================================
try:
    print("🔄 Esperando datos del ESP8266...")
    client.loop_forever()
except KeyboardInterrupt:
    print("🛑 Conexión finalizada por el usuario")
    client.disconnect()


✅ Modelo cargado correctamente


/tmp/ipython-input-1493549525.py:103: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


Streaming output truncated to the last 5000 lines.
12:13:06 | Prob: 0.6550 | Estado: ✅ Sin caída
12:13:07 | Prob: 0.6561 | Estado: ✅ Sin caída
12:13:07 | Prob: 0.6560 | Estado: ✅ Sin caída
12:13:07 | Prob: 0.6547 | Estado: ✅ Sin caída
12:13:07 | Prob: 0.6560 | Estado: ✅ Sin caída
12:13:08 | Prob: 0.6546 | Estado: ✅ Sin caída
12:13:08 | Prob: 0.6555 | Estado: ✅ Sin caída
12:13:08 | Prob: 0.6551 | Estado: ✅ Sin caída
12:13:08 | Prob: 0.6551 | Estado: ✅ Sin caída
12:13:09 | Prob: 0.6551 | Estado: ✅ Sin caída
12:13:09 | Prob: 0.6530 | Estado: ✅ Sin caída
12:13:09 | Prob: 0.6563 | Estado: ✅ Sin caída
12:13:09 | Prob: 0.6567 | Estado: ✅ Sin caída
12:13:10 | Prob: 0.6537 | Estado: ✅ Sin caída
12:13:10 | Prob: 0.6409 | Estado: ✅ Sin caída
12:13:10 | Prob: 0.6553 | Estado: ✅ Sin caída
12:13:10 | Prob: 0.6548 | Estado: ✅ Sin caída
12:13:11 | Prob: 0.6543 | Estado: ✅ Sin caída
12:13:11 | Prob: 0.6541 | Estado: ✅ Sin caída
12:13:11 | Prob: 0.6551 | Estado: ✅ Sin caída
12:13:11 | Prob: 0.6576 | Est